In [2]:
import requests
import pandas as pd
import numpy as np
import csv
from bs4 import BeautifulSoup

In [21]:
r = requests.get('https://www.worlddata.info/cost-of-living.php')
soup = BeautifulSoup(r.text, 'html.parser')

# world_data = [tag for tag in soup.find_all(['tr'])]
# soup.find_all('tr')[1].text.strip()[1:8]

arr = []
clean_arr = []
for i in range(101):
    j = soup.find_all(['tr'])[i].text.split()
    arr.append(j)

print(arr) 
  

world_data = [tag for tag in soup.find_all(['tr'])]
arr3 = []

n = 1
while n < 102:
    countries = world_data[n].a.text
    n+=1
    arr3.append(countries)

print(arr3)


[['RankCountry/RegionCost', 'indexØ', 'Monthly', 'incomePurchasing', 'power', 'index'], ['1Bermuda', '*157.910,437', 'USD103.8'], ['2Cayman', 'Islands', '*138.35,433', 'USD61.7'], ['3Switzerland131.47,454', 'USD89.1'], ['4Turks', 'and', 'Caicos', 'Islands', '*124.62,013', 'USD25.4'], ['5Israel123.14,554', 'USD58.1'], ['6Barbados121.81,613', 'USD20.8'], ['7Iceland120.15,685', 'USD74.3'], ['8Ireland110.66,756', 'USD96.0'], ['9Denmark109.26,100', 'USD87.8'], ['10Norway108.17,959', 'USD115.7'], ['11Australia107.05,036', 'USD74.0'], ['12Bahamas105.52,628', 'USD39.1'], ['13Vanuatu103.8297', 'USD4.5'], ['14Luxembourg103.47,600', 'USD115.5'], ['15New', 'Zealand101.44,038', 'USD62.6'], ['16Canada101.34,413', 'USD68.5'], ['17United', 'States100.06,364', 'USD100.0'], ['18United', 'Kingdom98.54,074', 'USD65.0'], ['19Finland95.64,530', 'USD74.5'], ['20Sweden93.95,249', 'USD87.9'], ['21Iran92.5325', 'USD5.5'], ['22Netherlands88.24,786', 'USD85.3'], ['23Belgium86.84,058', 'USD73.5'], ['24Japan84.93,5

**data from:** 

***https://visaguide.world/visa-free-countries/salvadoran-passport/***

***https://www.worlddata.info/cost-of-living.php***




In [18]:
# print(world_data[i:i].find_all(['td'])[1].text.strip())
# world_data_copy = world_data
# print(world_data[1].text)

world_data = [tag for tag in soup.find_all(['tr'])]
arr3 = []

n = 1
while n < 102:
    countries = world_data[n].a.text
    n+=1
    arr3.append(countries)

print(arr3)

# arr.append(world_data[1].a.text.strip())
# print(arr)

# list(soup.children)
    
# for f in world_data[1].findAll(['td'])[1].text.strip()

# countries = []
# for i in world_data_copy:
#     h = str(world_data[int(i)].a.text.strip())
#     countries.append(h)


['Bermuda', 'Cayman Islands', 'Switzerland', 'Turks and Caicos Islands', 'Israel', 'Barbados', 'Iceland', 'Ireland', 'Denmark', 'Norway', 'Australia', 'Bahamas', 'Vanuatu', 'Luxembourg', 'New Zealand', 'Canada', 'United States', 'United Kingdom', 'Finland', 'Sweden', 'Iran', 'Netherlands', 'Belgium', 'Japan', 'France', 'Austria', 'Germany', 'Italy', 'South Korea', 'Hong Kong', 'Spain', 'Singapore', 'Qatar', 'Portugal', 'United Arab Emirates', 'Macao', 'Greece', 'Czechia', 'Costa Rica', 'Lithuania', 'Mexico', 'Chile', 'China', 'Haiti', 'Croatia', 'Hungary', 'Brazil', 'Ecuador', 'Oman', 'El Salvador', 'Serbia', 'Poland', 'Albania', 'Brunei', 'Montenegro', 'Bulgaria', 'Nigeria', 'Romania', 'South Africa', 'Saudi Arabia', 'Mauritius', 'East Timor', 'Russia', 'Morocco', 'Kenya', 'Colombia', 'Cameroon', 'Georgia', 'Philippines', 'Lesotho', 'Nicaragua', 'Iraq', 'Bangladesh', 'Cambodia', 'Malaysia', 'Mongolia', 'Thailand', 'Bolivia', 'Indonesia', 'Tanzania', 'Vietnam', 'Kazakhstan', 'Benin', '

In [ ]:
r = requests.get('https://visaguide.world/visa-free-countries/salvadoran-passport/')
soup = BeautifulSoup(r.text, 'html.parser')

print(soup.find('ul'))